In [ ]:
import pandas as pd
import requests
import time

from etl_resources import sqlite_connection, get_api_key, get_symbol_list

In [ ]:
def build_prices_dataset():
    
    '''
    This function uses the alphavantage api to pull pricing information for 
    S&P 100 equities on a weekly basis. Weekly data has a much longer history.
    The dataframes are transposed and written to the local sqlite database for
    future use.
    '''
    
    api_key = get_api_key()
    symbol_list = get_symbol_list()
    con = sqlite_connection()
    
    for symbol in symbol_list:
        
        time.sleep(10) # Handle the rate limiter
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symbol}&apikey={api_key}"
        
        try:
            
            print(f"Parsing {symbol}")
            response = requests.get(url)
            json = response.json()['Weekly Time Series']

            df = pd.DataFrame.from_dict(json, orient='index')
            df.columns = [col.split(' ')[1] for col in df.columns]
            df = df.reset_index()
            df.rename(columns = {'index': 'date'}, inplace = True)
            df['ticker'] = symbol

            df.to_sql(name='weekly_prices', if_exists='append', index=False, con=con)
        
        except:
            print(f"Failed: {url}")


In [ ]:
build_prices_dataset()